<a href="https://colab.research.google.com/github/Pales23/Pales23/blob/main/pales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title - Instant (Advanced)
import requests
import os
import subprocess
cek = subprocess.getoutput('nvidia-smi')
if not 'T4' in cek:
  print("[1;32m-----Your colab session is over. Use other google account or wait 12-24 hours-----")
  print("-----Sesi colab elu udah habis bro. Saran gw tunggu 12-24 jam atau bikin/pake akun google lain-----")
  raise KeyboardInterrupt
from google.colab.output import eval_js
os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")

#thanks to https://github.com/camenduru for the fork
print('[1;32mInstalling dependencies...')
print('[1;32m(This process will only happen once per session)')
print('[0m')
%cd /content
#!if ! [ -f xformers ]; then pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl; pip install -q --pre triton; touch xformers; fi
if not os.path.isfile('dependencies'):
    !gdown http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
    !gdown https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
    !gdown https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
    !gdown https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
    !apt install -qq libunwind8-dev
    !dpkg -i *.deb
    %env LD_PRELOAD=libtcmalloc.so
    !rm *.deb
    print("Grabbing pre-built xformers from camenduru's repo")
    url = 'https://api.github.com/repos/camenduru/stable-diffusion-webui-colab/releases/latest'
    response = requests.get(url)
    print("Status code: " + str(response.status_code))
    if response.status_code == 200:
        data = response.json()
        templink = [link for link in [asset['browser_download_url'] for asset in data['assets'] if 'cp39' in asset['name']]]
        xformerlink = templink[0]
    else:
        print("Request failed, loading last updated xformers instead")
        xformerlink = 'https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.17/xformers-0.0.17+b6be33a.d20230315-cp39-cp39-linux_x86_64.whl'
    # !pip install -q --pre xformers
    !pip install -q  torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
    # !pip install -q --pre xformers==0.0.17.dev476 -U
    !pip install -q --pre triton
    !touch dependencies

print('[1;32mInstalling webui...')
print('[0m')
!if ! [ -d stable-diffusion-webui ]; then git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui; touch forkedwebui; fi
%cd stable-diffusion-webui

#AnythingV3 Model & Vae
print('[1;32mDownloading AnythingV3 model...')
print('[0m')
%cd /content/stable-diffusion-webui/models/Stable-diffusion/
!if ! [ -f Anything-V3.0.safetensors ]; then gdown https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp32-pruned.safetensors -O /content/stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0.safetensors; fi
!if ! [ -f Anything-V3.0.vae.pt ]; then gdown https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -O /content/stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0.vae.pt; fi

#webui setup for nai
print('[1;32mSetting UI configs...')
print('[0m')
%cd /content/stable-diffusion-webui
!if [ -f configinstanlite ]; then rm configinstanlite && rm /content/stable-diffusion-webui/modules/ui.py && mv /content/stable-diffusion-webui/modules/ui_backup.py /content/stable-diffusion-webui/modules/ui.py; fi
!if ! [ -f configinstan ]; then gdown https://pastebin.com/raw/gMTvTUUZ -O /content/stable-diffusion-webui/ui-config.json && gdown https://pastebin.com/raw/VviKmfiQ -O /content/stable-diffusion-webui/config.json; fi
!touch configinstan

#notification bell
!gdown https://github.com/etherealxx/etherportal-webui-colab/raw/master/notification.mp3 -O /content/stable-diffusion-webui/notification.mp3

#foolhardy remacri
! [ ! -d models/ESRGAN ] && mkdir -p models/ESRGAN; [ ! -f models/ESRGAN/4x_foolhardy_Remacri.pth ] && gdown https://huggingface.co/etherealxx/whoseisthis/resolve/main/4x_foolhardy_Remacri.pth -O /content/stable-diffusion-webui/models/ESRGAN/4x_foolhardy_Remacri.pth

#extensions
print('[1;32mInstalling extensions...')
print('[0m')
%cd /content/stable-diffusion-webui/extensions
!bash -c 'for repo in AlUlkesh/stable-diffusion-webui-images-browser Extraltodeus/depthmap2mask DominikDoom/a1111-sd-webui-tagcomplete ddPn08/kohya-sd-scripts-webui arenatemp/stable-diffusion-webui-model-toolkit etherealxx/batchlinks-webui; do if [ ! -d "$repo" ]; then git clone "https://github.com/$repo" && true; fi; done'
!git clone -b lora --single-branch https://github.com/etherealxx/sd-civitai-browser
!git clone -b pre-embedding --single-branch https://github.com/etherealxx/Umi-AI
#%cd /content/stable-diffusion-webui/extensions/UnivAICharGen
#!git checkout c56b5146049b91cfcd645c62bd356bd0631c4131 .

#badprompt & badartist
%cd /content/stable-diffusion-webui/embeddings/
!if ! [ -f bad_prompt.pt ]; then gdown https://huggingface.co/datasets/Nerfgun3/bad_prompt/resolve/main/bad_prompt_version2.pt -O /content/stable-diffusion-webui/embeddings/bad_prompt.pt; fi
!if ! [ -f bad_artist.pt ]; then gdown https://huggingface.co/NiXXerHATTER59/bad-artist/resolve/main/bad-artist.pt -O /content/stable-diffusion-webui/embeddings/bad_artist.pt; fi
!if ! [ -f bad_hands.pt ]; then gdown https://huggingface.co/etherealxx/whoseisthis/resolve/main/bad-hands-5.pt -O /content/stable-diffusion-webui/embeddings/bad_hands.pt; fi

#some fix
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' /content/stable-diffusion-webui/requirements_versions.txt

print('[1;32mInstallation Success!')
print('[1;32mLaunching the webui...')
print('[0m')
%cd /content/stable-diffusion-webui
!python launch.py --ckpt /content/stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0.safetensors --vae-path /content/stable-diffusion-webui/models/Stable-diffusion/Anything-V3.0.vae.pt --share --xformers --enable-insecure-extension-access --gradio-queue

Installing dependencies...
(This process will only happen once per session)

/content
Downloading...
From: http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
To: /content/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
100% 204k/204k [00:00<00:00, 758kB/s]
Downloading...
From: https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
To: /content/google-perftools_2.5-2.2ubuntu3_all.deb
100% 294k/294k [00:00<00:00, 903kB/s]
Downloading...
From: https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
To: /content/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
100% 91.6k/91.6k [00:00<00:00, 535kB/s]
Downloading...
From: https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
To: /content/libgoogle-perftools4_2.5-2.2ub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')